In [26]:
import pandas as pd
#!pip install xgboost
#from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
import seaborn as sns
import sklearn
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.model_selection import StratifiedKFold
from scipy.stats import norm
import numpy as np 


In [3]:
data=pd.read_csv('file4.csv')

In [4]:
data.head()

,Unnamed: 0,Customer,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,...,category_Oregon_Rural,category_Oregon_Suburban,category_Oregon_Urban,category_Washington_Rural,category_Washington_Suburban,category_Washington_Urban,category_Agent,category_Branch,category_Call Center,category_Web
0,0,BU79786,2763.519279,56274,69,32,5,0,1,384.811147,...,0,0,0,0,1,0,1,0,0,0
1,1,QZ44356,6979.535903,0,94,13,42,0,8,1131.464935,...,0,0,0,0,0,0,1,0,0,0
2,2,AI49188,12887.431650,48767,108,18,38,0,2,566.472247,...,0,0,0,0,0,0,1,0,0,0
3,3,WW63253,7645.861827,0,106,18,65,0,7,529.881344,...,0,0,0,0,0,0,0,0,1,0
4,4,HB64268,2813.692575,43836,73,12,44,0,1,138.130879,...,0,0,0,1,0,0,1,0,0,0


In [8]:
data=data.drop(['Unnamed: 0'],axis=1)

In [9]:
data.columns

Index(['Customer', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount',
       'CLV Compl', 'category_Basic', 'category_Extended', 'category_Premium',
       'category_Bachelor', 'category_College', 'category_Doctor',
       'category_High School or Below', 'category_Master', 'category_Disabled',
       'category_Employed', 'category_Medical Leave', 'category_Retired',
       'category_Unemployed', 'category_Divorced', 'category_Married',
       'category_Single', 'category_Corporate L1', 'category_Corporate L2',
       'category_Corporate L3', 'category_Personal L1', 'category_Personal L2',
       'category_Personal L3', 'category_Special L1', 'category_Special L2',
       'category_Special L3', 'category_Four-Door Car_Vehicle Size',
       'category_Luxury Car_Vehicle Size', 'category_Luxury SUV_Vehicle Size',
       'category_SUV_

In [17]:
new_data = data[data.columns]
new_data=new_data.drop(['Customer Lifetime Value','Customer'],axis=1)
X = new_data
y = data['Customer Lifetime Value']

In [18]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y ,test_size=0.2)
X_train.head()

,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,CLV Compl,category_Basic,category_Extended,...,category_Oregon_Rural,category_Oregon_Suburban,category_Oregon_Urban,category_Washington_Rural,category_Washington_Suburban,category_Washington_Urban,category_Agent,category_Branch,category_Call Center,category_Web
3698,22828,106,11,54,0,2,508.800000,9502.126747,1,0,...,0,1,0,0,0,0,1,0,0,0
2445,63455,61,26,71,0,1,287.654819,3350.459083,1,0,...,0,0,1,0,0,0,0,1,0,0
8093,88178,70,32,78,0,2,336.000000,8687.976289,1,0,...,0,0,0,0,0,0,0,0,1,0
4557,72191,68,19,48,0,2,209.387509,5581.535709,1,0,...,0,0,0,0,0,0,0,0,1,0
6625,26057,74,3,63,0,2,523.888845,7457.398935,1,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
train_data,fitted_lambda = stats.boxcox(X_train['Monthly Premium Auto'])
X_train['Monthly Premium Auto']=train_data
# use lambda value to transform test data
test_data = stats.boxcox(X_test['Monthly Premium Auto'], fitted_lambda)
X_test['Monthly Premium Auto']=test_data

/home/ayantika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ayantika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [20]:
y_train_new= np.log(y_train)

In [21]:
def mean_absolute_percentage_error(y_test, predictions): 
    y_true, y_pred = np.array(y_test), np.array(predictions)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [22]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFECV
gdb = GradientBoostingRegressor()
selector1 = RFECV(gdb, step=1, cv=5,n_jobs =-1,scoring="r2",verbose = 2)
#start_time = timer(None)
selector1.fit(X_train, y_train_new)
#timer(start_time)
selected1 = X_train.columns[selector1.support_]
print(selected1)

Fitting estimator with 62 features.
Fitting estimator with 61 features.
Fitting estimator with 60 features.
Fitting estimator with 59 features.
Fitting estimator with 58 features.
Fitting estimator with 57 features.
Fitting estimator with 56 features.
Fitting estimator with 55 features.
Fitting estimator with 54 features.
Fitting estimator with 53 features.
Fitting estimator with 52 features.
Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Index(['Income', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Total Claim Amount', 'CLV Compl',
       'category_Basic', 'category_Extended', 'category_Premium',
       'category_Bachelor', 'category_College', '

In [30]:
trainSelected = X_train[selected1]
testSelected = X_test[selected1]
gdb.fit(trainSelected, y_train_new)
y_pred_final = np.exp(gdb.predict(testSelected))
print("R2 Score :",r2_score(y_pred_final,y_test))
print("mean_absolute_percentage_error :",mean_absolute_percentage_error(y_test, predictions))

R2 Score : 0.3463031353376389
mean_absolute_percentage_error : 9.78576310976683


In [28]:
from sklearn.linear_model import LinearRegression
from sklearn import tree, linear_model
from sklearn.model_selection import StratifiedKFold

regr = linear_model.LinearRegression()
selector1 = RFECV(regr, step=1, cv=5,n_jobs =-1,scoring="r2",verbose = 2)
#start_time = timer(None)
selector1.fit(X_train, y_train_new)
#timer(start_time)
selected1 = X_train.columns[selector1.support_]
trainSelected = X_train[selected1]
testSelected = X_test[selected1]
regr.fit(trainSelected, y_train_new)
y_pred = np.exp(regr.predict(testSelected))
print("R2 Score :",r2_score(y_pred,y_test))
print("mean_absolute_percentage_error :",mean_absolute_percentage_error(y_test, predictions))

Fitting estimator with 62 features.
Fitting estimator with 61 features.
Fitting estimator with 60 features.
R2 Score : -6.359458623896661
mean_absolute_percentage_error : 9.78576310976683


In [29]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
selector1 = RFECV(xgb, step=1, cv=5,n_jobs =-1,scoring="r2",verbose = 2)
#start_time = timer(None)
selector1.fit(X_train, y_train_new)
#timer(start_time)
selected1 = X_train.columns[selector1.support_]
trainSelected = X_train[selected1]
testSelected = X_test[selected1]

xgb.fit(trainSelected,y_train_new)
predictions = np.exp(xgb.predict(testSelected))
print("Explained_variance_score :",explained_variance_score(predictions,y_test))
print("R2 Score :",r2_score(y_test, predictions))
print("mean_absolute_percentage_error :",mean_absolute_percentage_error(y_test, predictions))

Fitting estimator with 62 features.
[16:00:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 61 features.
[16:00:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 60 features.
[16:00:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 59 features.
[16:00:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 58 features.
[16:00:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 57 features.
[16:00:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with

/home/ayantika/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:01:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Explained_variance_score : 0.39798258910629813
R2 Score : 0.6787547785775804
mean_absolute_percentage_error : 9.78576310976683


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
selector1 = RFECV(regressor, step=1, cv=5,n_jobs =-1,scoring="r2",verbose = 2)
#start_time = timer(None)
selector1.fit(X_train, y_train_new)
#timer(start_time)
selected1 = X_train.columns[selector1.support_]
print(selected1)
trainSelected = X_train[selected1]
testSelected = X_test[selected1]
regressor.fit(trainSelected, y_train_new)
y_pred_final = np.exp(regressor.predict(testSelected))
r2_score(y_pred_final,y_test)